# Simulated PIK statistics

Here we inspect the accuracy and characteristics of the PIKs assigned,
leveraging our knowledge of ground truth from pseudopeople.

It wouldn't be possible to do the ground truth part with the real PVS, but
Layne, Wagner, and Rothhaas did something similar by redacting SSN from real records,
sending them through PVS without the SSN, and then using the true SSN
as ground truth.
The health care records they used are probably quite different from a CUF,
but they found a **very** good overall PIK accuracy (see cell below).

In [1]:
# Query planning is now on by default, but it has some rough edges.
# See https://github.com/dask/dask/issues/10995 for general discussion
# and https://github.com/dask/dask-expr/issues/1060 for the particular
# issue I ran into.
import dask
dask.config.set({"dataframe.query-planning": False})

In [2]:
import datetime, os, time

from vivarium_research_prl import distributed_compute, utils
from IPython.display import display

In [3]:
print(datetime.datetime.now())

2024-05-24 07:26:32.677296


In [4]:
# DO NOT EDIT if this notebook is not called ground_truth_accuracy.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
simulated_data_output_dir = 'output/generate_simulated_data'
case_study_output_dir = 'output'

# The "compute engine" is what we use on the Python side
# for our case-study-specific operations,
# as opposed to the Splink engine
compute_engine = 'pandas'
# Only matter if using a distributed compute engine
compute_engine_num_jobs = 3
compute_engine_cpus_per_job = 2
compute_engine_memory_per_job = "5GB"
queue = "long.q"
local_directory = f"/tmp/{os.environ['USER']}_{int(time.time())}_dask"

In [5]:
# Parameters
data_to_use = "usa"
simulated_data_output_dir = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study/generate_simulated_data/"
case_study_output_dir = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study/results/"
compute_engine = "dask"
compute_engine_num_jobs = 50
compute_engine_memory_per_job = "120GB"
compute_engine_cpus_per_job = 2
local_directory = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study_tmp/dask/"


In [6]:
if compute_engine == 'dask':
    utils.ensure_empty(local_directory)

In [7]:
case_study_output_dir = f'{case_study_output_dir}/{data_to_use}'
simulated_data_output_dir = f'{simulated_data_output_dir}/{data_to_use}'

In [8]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=compute_engine_num_jobs,
    cpus_per_job=compute_engine_cpus_per_job,
    memory_per_job=compute_engine_memory_per_job,
    queue=queue,
    local_directory=local_directory,
)

<Client: 'tcp://10.158.111.40:41733' processes=50 threads=50, memory=5.46 TiB>

In [9]:
census_2030_piked = df_ops.read_parquet(f'{case_study_output_dir}/census_2030_piked.parquet')
confirmed_piks_with_ground_truth = df_ops.read_parquet(f'{case_study_output_dir}/confirmed_piks.parquet')

In [10]:
piked_proportion = df_ops.compute(census_2030_piked.pik.notnull().mean())
# Compare with 90.28% of input records PIKed in the 2010 CUF,
# as reported in Wagner and Layne, Table 2, p. 18 
print(f'{piked_proportion:.2%} of the input records were PIKed')

89.49% of the input records were PIKed


In [11]:
# Multiple Census rows assigned the same PIK, indicating the model thinks they are duplicates in Census
pik_sizes = df_ops.persist(df_ops.groupby_agg_small_groups(census_2030_piked, by='pik', agg_func=lambda x: x.size()))
df_ops.compute(pik_sizes.value_counts())

1    305481683
2       704594
3          914
4           13
Name: count, dtype: int64

In [12]:
# Interesting: in pseudopeople, sometimes siblings are assigned the same (common) first name, making them almost identical.
# The only giveaway is their age and DOB.
# Presumably, this tends not to happen in real life.
duplicate_piks = pik_sizes.rename('pik_size').reset_index().pipe(lambda df: df[df.pik_size > 1])

df_ops.head(census_2030_piked.merge(duplicate_piks, on="pik").sort_values('pik'))

,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,record_id,pik,pik_size
2716,3632_492800,Amir,M,Lopez Ponce,2,0|/1G/2028,7323,s pleasant 2,<NA>,hillsboro,OR,97058,Household,Biological child,Male,Latino,2030,simulated_census_2030_125_986313,100_10000288,2
2731,3632_492800,Amanda,M,Lopez Ponce,26,01/16/2028,7323,s pleasant 2,<NA>,hillsboro,OR,97058,Household,Reference person,Female,Latino,2030,simulated_census_2030_125_776710,100_10000288,2
404,3978_12573,Sam,D,Thompson,57,09/21/1941,19,beryl street,<NA>,new york,NY,11236,Household,Other relative,Male,Black,2030,simulated_census_2030_137_26611,100_10000733,2
3853,3978_12573,Carl,D,Thompson,88,09/21/1941,19,beryl street,<NA>,new york,NY,11236,Household,Reference person,Male,Black,2030,simulated_census_2030_137_26608,100_10000733,2
1792,3465_41355,Samara,N,Cooley,12,09/19/2017,5014,242nd st sw,flat 5204,burbank,CA,94589,Household,Biological child,Female,Latino,2030,simulated_census_2030_111_86813,100_10001394,2
2455,3465_41355,Gretchen,N,Cooley,50,09/19/2017,5014,242nd st sw,flat 5204,burbank,CA,94589,Household,Opposite-sex spouse,Female,Latino,2030,simulated_census_2030_111_86810,100_10001394,2
1083,7817_307328,Dylan,E,Gaines,12,06/17/2017,<NA>,fox bnd ct,<NA>,grand blanc,MI,49657,Household,Sibling,Male,White,2030,simulated_census_2030_266_644676,100_10001446,2
2582,7817_535954,Dylan,E,Gaines,12,06/17/2017,615,sisson st,<NA>,fw,NaN,46268,Household,Other relative,Male,White,2030,simulated_census_2030_266_1025393,100_10001446,2
2003,5548_619778,Richard,D,Martinez,75,04/19/1954,122,sea forest dr,<NA>,albany,NY,14845,Household,Reference person,Male,Latino,2030,simulated_census_2030_185_229434,100_10001642,2
3992,2965_369050,Richard,D,Martinez,75,04/19/1954,11561,highland avenue,<NA>,<NA>,OR,97333,Household,Reference person,Male,Latino,2030,simulated_census_2030_91_774182,100_10001642,2


## Ground truth statistics

In [13]:
census_2030_ground_truth = df_ops.persist(
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_census_2030_ground_truth.parquet')
)

In [14]:
# In this version of pseudopeople, there are no actual duplicates in Census,
# which means all of the duplicates identified above are wrong.
assert len(census_2030_ground_truth) == len(df_ops.drop_duplicates(census_2030_ground_truth))

In [15]:
reference_files_ground_truth = df_ops.persist(df_ops.concat([
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_geobase_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_name_dob_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
], ignore_index=True))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    8.720000e+02
mean     4.181848e+07
std      5.982166e+04
min      4.164159e+07
25%      4.177921e+07
50%      4.182007e+07
75%      4.185967e+07
max      4.199184e+07
dtype: float64
Creating partitions of 729MB


In [16]:
# However, there can be reference file records that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(reference_files_ground_truth, by='record_id', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1    1229004194
2      59018942
3       1771457
4        102373
5          6077
6           351
7            16
8             1
9             1
Name: count, dtype: int64

In [17]:
reference_files_ground_truth = df_ops.persist(reference_files_ground_truth.merge(
    n_unique_simulants,
    on='record_id',
    how='left',
))
reference_files_ground_truth.head(n=100)

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_0_1010897,1559_19394,1
1,simulated_geobase_reference_file_0_1021693,6203_718056,1
2,simulated_geobase_reference_file_0_1028097,6793_444873,1
3,simulated_geobase_reference_file_0_1047685,6554_270819,1
4,simulated_geobase_reference_file_0_1057134,9495_189946,2
...,...,...,...
95,simulated_geobase_reference_file_0_2452588,2808_1138496,1
96,simulated_geobase_reference_file_0_246654,5831_663211,1
97,simulated_geobase_reference_file_0_2568436,6090_393091,1
98,simulated_geobase_reference_file_0_258644,4922_953730,1


In [18]:
df_ops.head(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == df_ops.compute(reference_files_ground_truth.n_unique_simulants.max())])

,record_id,simulant_id,n_unique_simulants
2185016,simulated_geobase_reference_file_102_1340703,6554_95394,9
2185017,simulated_geobase_reference_file_102_1340703,6554_95393,9
2185018,simulated_geobase_reference_file_102_1340703,6554_95397,9
2185019,simulated_geobase_reference_file_102_1340703,6554_95403,9
2185020,simulated_geobase_reference_file_102_1340703,6554_95389,9
2185021,simulated_geobase_reference_file_102_1340703,6554_95392,9
2185022,simulated_geobase_reference_file_102_1340703,6554_95396,9
2185023,simulated_geobase_reference_file_102_1340703,6554_95395,9
2185024,simulated_geobase_reference_file_102_1340703,6554_95406,9


In [19]:
census_2030_ground_truth = df_ops.persist(census_2030_ground_truth.merge(
    df_ops.drop_duplicates(reference_files_ground_truth[['simulant_id']]).assign(possible_to_pik=1),
    on='simulant_id',
    how='left',
).assign(possible_to_pik=lambda df: df.possible_to_pik.fillna(0)))
possible_to_pik_proportion = df_ops.compute(census_2030_ground_truth.possible_to_pik.mean())
print(
    f'{(1 - possible_to_pik_proportion):.2%} of the input records are '
    'impossible to PIK correctly, since they are not in any reference files'
)

0.43% of the input records are impossible to PIK correctly, since they are not in any reference files


In [20]:
print(
    f'Assigned PIKs to {(piked_proportion / possible_to_pik_proportion):.2%} of PIK-able records'
)

Assigned PIKs to 89.87% of PIK-able records


In [21]:
reference_file = df_ops.concat([
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_geobase_reference_file.parquet',
    ),
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_name_dob_reference_file.parquet',
    ),
], ignore_index=True)

In [22]:
reference_file_piks = df_ops.persist(reference_file[['record_id', 'pik']])
reference_file_piks

,record_id,pik
npartitions=358,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [23]:
assert len(reference_file_piks) == len(df_ops.drop_duplicates(reference_file_piks[['record_id']]))

In [24]:
pik_simulant_pairs = df_ops.persist(df_ops.drop_duplicates(reference_files_ground_truth.merge(reference_file_piks, on='record_id')[['pik', 'simulant_id']]))

In [25]:
# However, there can be PIKs that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(pik_simulant_pairs, by='pik', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1      353923759
2       41184067
3        3257802
4         225179
5          14963
         ...    
144            1
246            1
133            1
141            1
217            1
Name: count, Length: 335, dtype: int64

In [26]:
pik_simulant_pairs = df_ops.persist(pik_simulant_pairs.merge(
    n_unique_simulants,
    on='pik',
    how='left',
))
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=740,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [27]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == df_ops.compute(pik_simulant_pairs.n_unique_simulants.max())])

,pik,simulant_id,n_unique_simulants
552933,93_9726622,1655_508547,335
552934,93_9726622,7264_376391,335
552935,93_9726622,4123_481173,335
552936,93_9726622,6520_143952,335
552937,93_9726622,1282_490678,335
552938,93_9726622,7850_493685,335
552939,93_9726622,6144_432000,335
552940,93_9726622,3377_480936,335
552941,93_9726622,8997_280916,335
552942,93_9726622,1897_265557,335


## Definitions of accuracy

1. (most strict) Assigning any PIK with multiple simulants is incorrect
2. Assigning a PIK with multiple simulants is neither incorrect nor correct (excluded from denominator)
3. (most lenient) Assigning a PIK with multiple simulants is correct, as long as at least one of those simulants matches the truth

In [28]:
# All modules, Medicare database, calculated from Layne, Wagner, and Rothhaas Table 1 (p. 15)
real_life_pvs_accuracy = 1 - (2_585 + 60_709 + 129_480 + 89_094) / (52_406_981 + 5_170_924 + 49_374_794 + 50_327_034)
f'{real_life_pvs_accuracy:.5%}'

'99.82079%'

### Definition 1

In [29]:
piks_assigned = df_ops.compute(census_2030_piked.pik.notnull().sum())
piks_assigned

306893665

In [30]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants > 1])

,pik,simulant_id,n_unique_simulants
4,100_10001131,1299_577845,2
5,100_10001131,7344_252780,2
12,100_10005841,2787_347774,2
13,100_10005841,2787_692084,2
14,100_10006089,8869_1057428,2
15,100_10006089,8869_714641,2
25,100_10013375,3984_1041763,2
26,100_10013375,3984_271827,2
77,100_10044433,3465_567846,2
78,100_10044433,8134_1070213,2


In [31]:
single_sim_piks_correct = df_ops.compute(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_piks_correct

270242236

In [32]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / piks_assigned
)

0.8805728720402228

In [33]:
assert len(confirmed_piks_with_ground_truth) == piks_assigned

In [34]:
df_ops.head(census_2030_ground_truth.rename(columns={'record_id': 'record_id_census_2030'}))

,record_id_census_2030,simulant_id,possible_to_pik
0,simulated_census_2030_0_428,28_512,1.0
1,simulated_census_2030_0_1517,28_1781,1.0
2,simulated_census_2030_0_2703,28_3203,1.0
3,simulated_census_2030_0_3657,28_4338,1.0
4,simulated_census_2030_0_4661,28_5528,1.0
5,simulated_census_2030_0_4773,28_5659,1.0
6,simulated_census_2030_0_4791,28_5679,1.0
7,simulated_census_2030_0_5737,28_6800,1.0
8,simulated_census_2030_0_6401,28_7571,1.0
9,simulated_census_2030_0_6800,28_8045,1.0


In [35]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_correct = df_ops.compute(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_record_links_correct

287197731

In [36]:
(
    single_sim_record_links_correct / piks_assigned
)

0.9358216338548403

### Definition 2

In [37]:
single_sim_piks_assigned = len(census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == 1][['pik', 'simulant_id']]))
single_sim_piks_assigned

270448733

In [38]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / single_sim_piks_assigned
)

0.9992364652712202

In [39]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_assigned = df_ops.compute(
    (confirmed_piks_with_ground_truth
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .n_unique_simulants == 1).sum()
)
single_sim_record_links_assigned

287419292

In [40]:
(
    single_sim_record_links_correct / single_sim_record_links_assigned
)

0.9992291366440357

### Definition 3

In [41]:
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=740,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [42]:
piks_at_least_partially_correct = df_ops.persist(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(df_ops.drop_duplicates)
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id", "pik"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
piks_at_least_partially_correct

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.480000e+03
mean     2.654147e+07
std      4.395391e+07
min      0.000000e+00
25%      0.000000e+00
50%      1.454150e+04
75%      2.244691e+07
max      2.658089e+08
dtype: float64
Creating partitions of 786MB


,record_id,pik,correct
npartitions=54,,,
,string,string,bool[pyarrow]
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [43]:
# Overall accuracy, treating it as a black box
piks_correct_proportion = (df_ops.compute(piks_at_least_partially_correct.correct.sum()) / piks_assigned)
piks_correct_proportion

0.9992483976493943

In [44]:
print(f'{piks_correct_proportion:.5%} of the PIKs assigned were correct; compare with {real_life_pvs_accuracy:.5%} in real life')

99.92484% of the PIKs assigned were correct; compare with 99.82079% in real life


In [45]:
# Looking at whether the exact *record* linked was from the same simulant
sim_record_links_at_least_partially_correct = df_ops.persist(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id_raw_input_file", "record_id_reference_file", "pik", "module_name", "pass_name"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
sim_record_links_at_least_partially_correct

,record_id_raw_input_file,record_id_reference_file,pik,module_name,pass_name,correct
npartitions=849,,,,,,
,string,string,string,string,string,bool[pyarrow]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [46]:
len(sim_record_links_at_least_partially_correct)

306893665

In [47]:
len(df_ops.drop_duplicates(sim_record_links_at_least_partially_correct[['record_id_raw_input_file', 'record_id_reference_file']]))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    1.698000e+03
mean     1.641192e+07
std      2.845312e+07
min      0.000000e+00
25%      0.000000e+00
50%      1.113800e+04
75%      1.022008e+07
max      1.561296e+08
dtype: float64
Creating partitions of 557MB


306893665

In [48]:
(
    df_ops.compute(sim_record_links_at_least_partially_correct.correct.sum()) / piks_assigned
)

0.9992451685178969

In [49]:
assert df_ops.compute((df_ops.groupby_agg_small_groups(confirmed_piks_with_ground_truth, by='record_id_raw_input_file', agg_func=lambda x: x.record_id_reference_file.nunique()) <= 1).all())

In [50]:
# Using definition 3 -- at the PIK level
piks_at_least_partially_correct = df_ops.persist(
    piks_at_least_partially_correct
        .rename(columns={'record_id': 'record_id_raw_input_file'})
        .merge(confirmed_piks_with_ground_truth[['record_id_raw_input_file', 'module_name', 'pass_name']], on='record_id_raw_input_file')
)
piks_at_least_partially_correct

,record_id_raw_input_file,pik,correct,module_name,pass_name
npartitions=849,,,,,
,string,string,bool[pyarrow],string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [51]:
# Accuracy by module -- note that this shows the opposite pattern (with the sample data)
# relative to the results of Layne et al., who found GeoSearch was much *more* accurate
df_ops.compute(piks_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
dobsearch,0.995207,157934
namesearch,0.997353,18703967
geosearch,0.999372,287235940
hhcompsearch,0.999913,795824


In [52]:
# Accuracy by pass -- could be used to tune pass-specific cutoffs, but
# this might not be too informative while we are still using the sample data.
df_ops.compute(piks_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

,,mean,size
module_name,pass_name,,
dobsearch,initials name switch,0.995207,157934
namesearch,DOB and NYSIIS of name,0.997353,18701962
geosearch,geokey name switch,0.998792,899891
namesearch,DOB and initials,0.999002,2005
geosearch,geokey,0.999266,242653619
hhcompsearch,year of birth,0.999865,400996
geosearch,house number and street name Soundex name switch,0.999957,46424
hhcompsearch,initials,0.999962,394828
geosearch,some name and DOB information,0.999974,30018460


In [53]:
# Using definition 3 -- at the link level
df_ops.compute(sim_record_links_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
dobsearch,0.995207,157934
namesearch,0.997353,18703967
geosearch,0.999369,287235940
hhcompsearch,0.999912,795824


In [54]:
df_ops.compute(sim_record_links_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

,,mean,size
module_name,pass_name,,
dobsearch,initials name switch,0.995207,157934
namesearch,DOB and NYSIIS of name,0.997353,18701962
geosearch,geokey name switch,0.998789,899891
namesearch,DOB and initials,0.999002,2005
geosearch,geokey,0.999262,242653619
hhcompsearch,year of birth,0.999865,400996
geosearch,house number and street name Soundex name switch,0.999957,46424
hhcompsearch,initials,0.999959,394828
geosearch,some name and DOB information,0.999974,30018460


In [55]:
df_ops.compute(sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct].groupby(["module_name", "pass_name"]).size()).sort_values()

module_name   pass_name                                       
namesearch    DOB and initials                                         2
geosearch     house number and street name Soundex name switch         2
hhcompsearch  initials                                                16
              year of birth                                           54
geosearch     house number and street name Soundex                   352
dobsearch     initials name switch                                   757
geosearch     some name and DOB information                          788
              geokey name switch                                    1090
namesearch    DOB and NYSIIS of name                               49509
geosearch     geokey                                              179083
dtype: int64

### Incorrect and missed PIKs

In [56]:
incorrectly_linked_pairs = df_ops.persist(df_ops.drop_duplicates(
    sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct]
        [["record_id_raw_input_file", "record_id_reference_file"]]
))
incorrectly_linked_pairs

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count     1698.000000
mean     12469.694935
std      12490.898424
min          0.000000
25%          0.000000
50%       9535.500000
75%      24951.250000
max      28393.000000
dtype: float64
Creating partitions of 100MB


,record_id_raw_input_file,record_id_reference_file
npartitions=1,,
,string,string
,...,...


In [57]:
len(incorrectly_linked_pairs)

231653

In [58]:
incorrect_links = df_ops.head(incorrectly_linked_pairs, n=100)
incorrect_links

,record_id_raw_input_file,record_id_reference_file
0,simulated_census_2030_0_1000428,simulated_geobase_reference_file_179_1002396
1,simulated_census_2030_0_1000771,simulated_geobase_reference_file_183_1468335
2,simulated_census_2030_0_1000862,simulated_geobase_reference_file_147_3743026
3,simulated_census_2030_0_1001175,simulated_name_dob_reference_file_17_7088846
4,simulated_census_2030_0_1001701,simulated_geobase_reference_file_21_2470024
...,...,...
95,simulated_census_2030_0_151396,simulated_geobase_reference_file_151_74598
96,simulated_census_2030_0_151692,simulated_geobase_reference_file_108_794912
97,simulated_census_2030_0_15224,simulated_geobase_reference_file_197_4064041
98,simulated_census_2030_0_152813,simulated_name_dob_reference_file_17_4241690


In [59]:
%xdel incorrectly_linked_pairs

In [60]:
comparison_cols = [
    "first_name",
    "middle_name",
    "last_name",
    "date_of_birth",
    "street_number",
    "street_name",
    "unit_number",
    "city",
    "state",
]

incorrect_links_detail = (
    incorrect_links
        .merge(
            df_ops.compute(census_2030_piked[census_2030_piked.record_id.isin(incorrect_links.record_id_raw_input_file)])
                .rename(columns={"record_id": "record_id_raw_input_file", "middle_initial": "middle_name"})
                [["record_id_raw_input_file"] + comparison_cols],
            on="record_id_raw_input_file",
            how="left",
        )
        .merge(
            df_ops.compute(reference_file[reference_file.record_id.isin(incorrect_links.record_id_reference_file)])
                .rename(columns={"record_id": "record_id_reference_file"})
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                [["record_id_reference_file"] + comparison_cols],
            on="record_id_reference_file",
            how="left",
            suffixes=("_census", "_reference_file"),
        )
)
def flatten(xss):
    return [x for xs in xss for x in xs]

incorrect_links_detail[flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])]

,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,Aurora,Dean,G,Gabriel,Harmon,Harmon,08/29/2028,20280829,5543,5543,highway 413,HIGHWAY 413,<NA>,<NA>,richmond,RICHMOND,VA,VA
1,Aurora,Luz,L,Linda,Devi,Devi,11/14/2022,20221114,2140,2140,hwy 88 w,HWY 88 W,<NA>,<NA>,winter park,WINTER PARK,FL,FL
2,<NA>,Benjamin,R,David,Gatian,Gatian,07/25/2024,20240725,606,606,lincoln woods drive,LINCOLN WOODS DRIVE,<NA>,<NA>,norfolk,NORFOLK,VA,VA
3,Mila,Mila,G,Gia,Garcia,Garcia,10/07/2028,20281007,503,<NA>,n villa st,<NA>,<NA>,<NA>,sanford,<NA>,ME,<NA>
4,Alaia,Martin,A,Alexander,Reitz,Reitz,10/12/2028,20281012,6100,6100,burdette drive,BURDETTE DRIVE,<NA>,<NA>,chicago,CHICAGO,IL,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Zachary,Michele,J,Julie,Goldsby,Goldsby,07/27/1966,19660727,17823-17835,17823-17835,polo pk dr,POLO PK DR,<NA>,<NA>,east peoria,EAST PEORIA,IL,IL
96,Kash,Carter,M,Man Of,Waters,Waters,02/24/2009,20090224,4243,4243,appian way w,APPIAN WAY W,<NA>,<NA>,ridgefield,RIDGEFIELD,NJ,NJ
97,Grant,Chad,B,<NA>,Nijjar,Nijjar,08/01/1943,19430801,1561,1561,hannalei pl,HANNALEI PL,<NA>,<NA>,martinez,MARTINEZ,CA,CA
98,Jackson,Jackson,B,Bodhi,Willis,Willis,12/04/2018,20181204,620,<NA>,picketts cove,<NA>,fl # 1 apt # 1010,<NA>,pine valley,<NA>,TX,<NA>


In [61]:
missed_links = df_ops.persist(
    census_2030_piked[census_2030_piked.pik.isnull()][["record_id"]]
        .merge(census_2030_ground_truth, on="record_id")
        .merge(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == 1], on="simulant_id", suffixes=("_census", "_reference_file"))
)

In [62]:
len(missed_links)

107488068

In [63]:
simulants_missed = df_ops.head(missed_links[['simulant_id']], n=100).simulant_id.unique()
simulants_missed

<ArrowStringArray>
[  '352_205030', '3454_1055553',  '1282_803576', '1482_1106062',
 '1648_1145213',  '8612_598382',  '9696_241917',   '440_818214',
 '5831_1163158',  '9643_779883', '2298_1157048',  '682_1102032',
  '9804_323231',  '4649_635246', '9911_1133450',   '278_343671',
  '7938_532559',  '6144_683864',  '9901_575637',   '6760_71186',
  '3142_100881',   '107_178364', '4950_1032017',  '1182_579036',
  '8869_544928', '9840_1053660',  '9740_349524',    '93_180423',
 '3607_1151651', '8425_1183995']
Length: 30, dtype: string

In [64]:
missed_pairs = df_ops.compute(missed_links[missed_links.simulant_id.isin(list(simulants_missed))])
missed_pairs

,record_id_census,simulant_id,possible_to_pik,record_id_reference_file,n_unique_simulants
0,simulated_census_2030_11_172699,352_205030,1.0,simulated_geobase_reference_file_35_3252330,1
1,simulated_census_2030_11_172699,352_205030,1.0,simulated_name_dob_reference_file_41_1554120,1
2,simulated_census_2030_11_172699,352_205030,1.0,simulated_geobase_reference_file_35_3252331,1
3,simulated_census_2030_110_892085,3454_1055553,1.0,simulated_geobase_reference_file_55_2082491,1
4,simulated_census_2030_110_892085,3454_1055553,1.0,simulated_name_dob_reference_file_26_3222629,1
...,...,...,...,...,...
96,simulated_census_2030_2_152033,93_180423,1.0,simulated_name_dob_reference_file_10_4494978,1
97,simulated_census_2030_122_979888,3607_1151651,1.0,simulated_geobase_reference_file_86_2343454,1
98,simulated_census_2030_122_979888,3607_1151651,1.0,simulated_name_dob_reference_file_29_5413790,1
99,simulated_census_2030_281_1010739,8425_1183995,1.0,simulated_name_dob_reference_file_42_1276576,1


In [65]:
%xdel missed_links

In [66]:
missed_links_detail = (
    missed_pairs
        .merge(
            df_ops.compute(census_2030_piked[census_2030_piked.record_id.isin(list(missed_pairs.record_id_census))])
                .rename(columns={"record_id": "record_id_census", "middle_initial": "middle_name"}),
            on="record_id_census",
        )
        .merge(
            df_ops.compute(reference_file[reference_file.record_id.isin(missed_pairs.record_id_reference_file)])
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                .rename(columns={"record_id": "record_id_reference_file"}),
            on="record_id_reference_file",
            suffixes=("_census", "_reference_file"),
        )
)

In [67]:
for simulant in simulants_missed:
    print(simulant)
    display(missed_links_detail[missed_links_detail.simulant_id == simulant][['simulant_id'] + flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])])

352_205030


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,352_205030,Shlomie,<NA>,D,Desean,Smallwood,Smallwood,22/10/1995,19951022,14308,14308,colby dr,<NA>,<NA>,<NA>,watertown,WATERTOWN,CT,CT
1,352_205030,Shlomie,<NA>,D,Desean,Smallwood,Smallwood,22/10/1995,19951022,14308,<NA>,colby dr,<NA>,<NA>,<NA>,watertown,<NA>,CT,<NA>
2,352_205030,Shlomie,<NA>,D,Desean,Smallwood,Smallwood,22/10/1995,19951022,14308,14308,colby dr,COLBY DR,<NA>,<NA>,watertown,WATERTOWN,CT,CT


3454_1055553


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
3,3454_1055553,Knowledge,Knowledge,B,Mrs,Bach,Bach,03/18/2022,20261803,<NA>,<NA>,logans run,LOGANS RUN,<NA>,<NA>,rhome,RHOME,TX,TX
4,3454_1055553,Knowledge,Knowledge,B,Mrs,Bach,Bach,03/18/2022,20261803,<NA>,<NA>,logans run,<NA>,<NA>,<NA>,rhome,<NA>,TX,<NA>


1282_803576


...

,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
95,93_180423,Quanesha,Quanesha,Z,Zoe,Lloyd,Lbyd,10/05/1991,19911005,2201,2201,wildwood drive,WILDWOOD DRIVE,<NA>,<NA>,wbarton,WHARTON,TX,TX
96,93_180423,Quanesha,Quanesha,Z,Zoe,Lloyd,Lbyd,10/05/1991,19911005,2201,<NA>,wildwood drive,<NA>,<NA>,<NA>,wbarton,<NA>,TX,<NA>


3607_1151651


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
97,3607_1151651,G,Owen,Natarajan,Giovanni,Owen,Natarajan,08/03/2027,20270803,2575,2575,co rd 16,CO RD 16,<NA>,<NA>,pukalani,PUKALANI,HI,HI
98,3607_1151651,G,Owen,Natarajan,Giovanni,Owen,Natarajan,08/03/2027,20270803,2575,<NA>,co rd 16,<NA>,<NA>,<NA>,pukalani,<NA>,HI,<NA>


8425_1183995


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
99,8425_1183995,Harper,Harper,H,Harper,Decotis,Decotis,053/08/2017,20170308,1065,<NA>,hillside av,<NA>,<NA>,<NA>,norman,<NA>,OK,<NA>
100,8425_1183995,Harper,Harper,H,Harper,Decotis,Decotis,053/08/2017,20170308,1065,5065,hillside av,HILLSIDE AV,<NA>,<NA>,norman,NORMAN,OK,OK
